# Few-shot JERX task offline inference

In [ ]:
#|default_exp jerx.offline.llm

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export
from pathlib import Path
import pandas as pd
from bellek.jerx.utils import parse_triplets
from bellek.logging import get_logger

log = get_logger(__name__)

In [ ]:
#|hide
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# |export


def parse_triplet_response(response: str, *args, **kwargs) -> list[tuple[str, str, str]]:
    triplets = parse_triplets(response.strip())
    return [(e1, rel, e2) if e1 != e2 else (e1, rel, e2 + "(obj)") for e1, rel, e2 in triplets]


def make_kg_triplet_extract_fn(inference_cache_filepath: Path):
    df = pd.read_json(inference_cache_filepath, lines=True)
    mapping = {text: generation for text, generation in zip(df["text"], df["generation"])}
    del df

    def extract_kg_triplets(text: str) -> list[tuple[str, str, str]]:
        return parse_triplet_response(mapping[text])

    return extract_kg_triplets

In [ ]:
#|hide
jerx_inferences_filepath = Path("../data/raw/musique-kg-llm/train/jerx-inferences/llama3-base.jsonl")

df = pd.read_json(jerx_inferences_filepath, lines=True)
df

,id,paragraph_idx,is_supporting,text,__index_level_0__,input,output,generation
0,2hop__128801_205185,1,True,"# Midland, Texas\nMidland is a city in and the...",0,[{'content': 'You are an excellent knowledge g...,[],Midland | location | Texas\nMidland | county s...
1,2hop__128801_205185,11,True,"# KNFM\nKNFM (92.3 FM), branded as ""Lonestar 9...",1,[{'content': 'You are an excellent knowledge g...,[],KNFM | frequency | 92.3 FM\nKNFM | broadcastin...
2,2hop__128806_205185,5,True,"# KQRX\nKQRX (95.1 FM, ""Rock 95.1"") is a comme...",2,[{'content': 'You are an excellent knowledge g...,[],KQRX | broadcast frequency | 95.1 FM\nKQRX | b...
3,2hop__128895_11424,3,True,"# Atlantic City, New Jersey\nThere were 15,504...",4,[{'content': 'You are an excellent knowledge g...,[],"Atlantic City, New Jersey | location | New Jer..."
4,2hop__128895_11424,12,True,# WPUR\nWPUR (107.3 FM) is a country music for...,5,[{'content': 'You are an excellent knowledge g...,[],"WPUR | location | Atlantic City, New Jersey\nW..."
...,...,...,...,...,...,...,...,...
78,2hop__651488_94210,12,True,# Uffizi\nThe building of Uffizi complex was b...,104,[{'content': 'You are an excellent knowledge g...,[],Uffizi | building architect | Giorgio Vasari\n...
79,2hop__651488_94210,18,True,# Pietà (Perugino)\nPietà is a painting by the...,105,[{'content': 'You are an excellent knowledge g...,[],Pietà | artist | Pietro Perugino\nPietà | year...
80,2hop__525596_543261,18,True,# Jim Norton (defensive lineman)\nJames Alfred...,107,[{'content': 'You are an excellent knowledge g...,[],Jim Norton (Athlete) | birth date | November 1...
81,2hop__415654_46042,0,True,# El Yunque National Forest\nEl Yunque Nationa...,108,[{'content': 'You are an excellent knowledge g...,[],El Yunque National Rainforest | location | Sie...


In [ ]:
#|hide
func = make_kg_triplet_extract_fn(jerx_inferences_filepath)
text = df.iloc[0]['text']
print(func(text))

[('Midland', 'location', 'Texas'), ('Midland', 'county seat', 'Midland County'), ('Midland', 'city', 'Midland(obj)'), ('Midland', 'region', 'Southern Plains'), ('Midland', 'area', 'Western')]


In [ ]:
row = df[df['id']=='2hop__145162_126070'].iloc[0]
row

id                                                 2hop__145162_126070
paragraph_idx                                                        7
is_supporting                                                     True
text                 # Leon Ichaso\nLeon Ichaso was born in Havana ...
__index_level_0__                                                   64
input                [{'content': 'You are an excellent knowledge g...
output                                                              []
generation           Leon Ichaso | birth date | August 3, 1948\nLeo...
Name: 53, dtype: object

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()